In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import animation  
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import manifold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
dao_data = pd.read_csv("tokens.csv")
dao_data = dao_data.copy().dropna()
dao_data.head(4)

,_id/$oid,name,symbol,contractAddress,chain/$oid,logo,decimal,financialData/price,financialData/volume24h,financialData/volumeChange24h,...,financialData/percentChange60d,financialData/percentChange90d,financialData/marketCap,financialData/marketCapDominance,financialData/fullyDilutedMarketCap,financialData/lastUpdated/$date,live,createdAt/$date,updatedAt/$date,__v
1,623e1d5fd02aba1aed362f99,Alchemix,ALCX,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,623c7bc416dc4a608cd24b00,https://s2.coinmarketcap.com/static/img/coins/...,18,107.118305,2.394103e+07,98.6279,...,-29.946043,-47.405427,1.281386e+08,0.0064,1.674200e+08,2022-03-25T19:50:00Z,True,2022-03-25T19:51:59.218Z,2022-03-25T19:51:59.218Z,0
2,623e30db29f0a800b9b76b4b,Aavegotchi,GHST,0x3F382DbD960E3a9bbCeaE22651E88158d2791550,623c7bc416dc4a608cd24b00,https://s2.coinmarketcap.com/static/img/coins/...,18,2.012771,2.222964e+07,-26.2485,...,0.515132,-15.602256,1.252114e+08,0.0063,1.287623e+08,2022-03-25T21:13:00Z,True,2022-03-25T21:15:07.758Z,2022-03-25T21:15:07.758Z,0
3,623e30dd29f0a800b9b76b4d,Decentraland,MANA,0x0f5d2fb29fb7d3cfee444a200298f468908cc942,623c7bc416dc4a608cd24b00,https://s2.coinmarketcap.com/static/img/coins/...,18,2.604602,4.620275e+08,-35.9028,...,25.014858,-30.904646,4.794600e+09,0.2399,5.713784e+09,2022-03-25T21:13:00Z,True,2022-03-25T21:15:09.820Z,2022-03-25T21:15:09.820Z,0
4,623e30df29f0a800b9b76b4f,Magic Ethereum Money,MEM,0x343e59d9d835e35b07fe80f5bb544f8ed1cd3b11,623c7bc416dc4a608cd24b00,https://s2.coinmarketcap.com/static/img/coins/...,18,0.000005,9.751028e+04,102.6711,...,37.165415,37.165415,0.000000e+00,0.0000,4.969091e+06,2022-03-25T21:13:00Z,True,2022-03-25T21:15:11.304Z,2022-03-25T21:15:11.304Z,0


In [3]:
dao_data.shape

(126, 24)

In [4]:
dao_data.pop("_id/$oid")
dao_data.pop("name")
dao_data.pop("symbol")
dao_data.pop("contractAddress")
dao_data.pop("chain/$oid")
dao_data.pop("logo")
dao_data.pop("decimal")
dao_data.pop("financialData/lastUpdated/$date")
dao_data.pop("live")
dao_data.pop("createdAt/$date")
dao_data.pop("updatedAt/$date")
dao_labels = dao_data.pop("__v")

In [5]:
dao_data.shape

(126, 12)

In [6]:
input_x = {}
for name, column in dao_data.items():
    dtype = column.dtype
    if dtype == object:
        dtype = tf.string
    else:
        dtype = tf.float32
    input_x[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

input_x

{'financialData/price': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/price')>,
 'financialData/volume24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/volume24h')>,
 'financialData/volumeChange24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/volumeChange24h')>,
 'financialData/percentChange1h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange1h')>,
 'financialData/percentChange24h': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange24h')>,
 'financialData/percentChange7d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange7d')>,
 'financialData/percentChange30d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData/percentChange30d')>,
 'financialData/percentChange60d': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'financialData

In [7]:
numeric_inputs = {name: input for name, input in input_x.items() if input.dtype == tf.float32}

x = tf.keras.layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(dao_data[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

all_numeric_inputs

<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'normalization')>

In [8]:
for name, input in input_x.items():
    if input.dtype != tf.string:
        continue
    
    lookup = preprocessing.StringLookup(vocabulary=np.unique(dao_data[name]))
    one_hot = preprocessing.CategoryEncoding(max_tokens=lookup.vocab_size())

#    x = tf.strings.lower(name)
#    x = tf.strings.strip(x)
    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x) 
    
preprocessed_inputs

[<KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'normalization')>]

In [9]:
preprocessing_layer = tf.keras.Model(input_x, preprocessed_inputs, name="Processed DAO Data")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [10]:
dao_features_dict = {name: np.array(value) for name, value in dao_data.items()}
two_dao_sample_dict = {name:values[1:3, ] for name, values in dao_features_dict.items()}
two_dao_sample_dict

{'financialData/price': array([2.01277126, 2.60460218]),
 'financialData/volume24h': array([2.22296429e+07, 4.62027472e+08]),
 'financialData/volumeChange24h': array([-26.2485, -35.9028]),
 'financialData/percentChange1h': array([-0.0839251 ,  0.08114252]),
 'financialData/percentChange24h': array([-0.81904903, -1.60651805]),
 'financialData/percentChange7d': array([2.86166081, 6.57703982]),
 'financialData/percentChange30d': array([ 1.96809949, -0.74814829]),
 'financialData/percentChange60d': array([ 0.51513232, 25.01485821]),
 'financialData/percentChange90d': array([-15.60225555, -30.90464572]),
 'financialData/marketCap': array([1.25211404e+08, 4.79460032e+09]),
 'financialData/marketCapDominance': array([0.0063, 0.2399]),
 'financialData/fullyDilutedMarketCap': array([1.28762262e+08, 5.71378412e+09])}

In [11]:
two_sample_fitted = preprocessing_layer(two_dao_sample_dict)
two_dao_sample_dict

{'financialData/price': array([2.01277126, 2.60460218]),
 'financialData/volume24h': array([2.22296429e+07, 4.62027472e+08]),
 'financialData/volumeChange24h': array([-26.2485, -35.9028]),
 'financialData/percentChange1h': array([-0.0839251 ,  0.08114252]),
 'financialData/percentChange24h': array([-0.81904903, -1.60651805]),
 'financialData/percentChange7d': array([2.86166081, 6.57703982]),
 'financialData/percentChange30d': array([ 1.96809949, -0.74814829]),
 'financialData/percentChange60d': array([ 0.51513232, 25.01485821]),
 'financialData/percentChange90d': array([-15.60225555, -30.90464572]),
 'financialData/marketCap': array([1.25211404e+08, 4.79460032e+09]),
 'financialData/marketCapDominance': array([0.0063, 0.2399]),
 'financialData/fullyDilutedMarketCap': array([1.28762262e+08, 5.71378412e+09])}

In [13]:
full_dim = two_sample_fitted.shape.as_list()[1]

encoding_dim1 = 128
encoding_dim2 = 16
encoding_dim3 = 3 

encoder_input_data = keras.Input(shape=(full_dim,))

encoded_layer1 = keras.layers.Dense(encoding_dim1, activation='relu')(encoder_input_data)
encoded_layer2 = keras.layers.Dense(encoding_dim2, activation='relu')(encoded_layer1)
encoded_layer3 = keras.layers.Dense(encoding_dim3, activation='relu', name="ClusteringLayer")(encoded_layer2)

encoder_model = keras.Model(encoder_input_data, encoded_layer3)

decoded_layer3 = keras.layers.Dense(encoding_dim2, activation='relu')(encoded_layer3)
decoded_layer2 = keras.layers.Dense(encoding_dim1, activation='relu')(decoded_layer3)
decoded_layer1 = keras.layers.Dense(full_dim, activation='sigmoid')(decoded_layer2)

autoencoder_model = keras.Model(encoder_input_data, outputs=decoded_layer1, name="Encoder")

autoencoder_model.compile(optimizer="RMSprop", loss=tf.keras.losses.mean_squared_error)
tf.keras.utils.plot_model(model=autoencoder_model, rankdir="LR", dpi=130, show_shapes=True, to_file="autoencoder.png")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
